In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import time
from datetime import date, timedelta
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import warnings
warnings.filterwarnings('ignore')

### Atualização de temporada

In [2]:
def clean_name(name):
    name = re.sub(r'\s*-.*$', '', name.strip())  # Remover informações extras
    return name.title()  # Caso contrário, padronizar nome com letras maiúsculas iniciais

In [3]:
def find_point_five(number):
    # Converte o número em uma string
    number_str = str(number)
    # Verifica se a string termina com ".5"
    if number_str.endswith(".5"):
        return True
    else:
        return False

In [4]:
hoje = date.today()
amanha = hoje + timedelta(days=1)

dia = "hoje"

## Scraping dos jogos do dia

In [5]:
try:
    if dia == 'hoje':
        existentes = pd.read_csv(f'jogos_do_dia/{hoje}.csv')
    else:
        existentes = pd.read_csv(f'jogos_do_dia/{amanha}.csv')
        
    existentes = existentes['Fixture ID'].unique().tolist()
except:
    existentes = []

In [6]:
# Instanciando o Objeto ChromeOptions
options = webdriver.EdgeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Edge(options=options)

# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com.br/basquete/")
time.sleep(2)

# Para jogos do dia seguinte
if dia == "amanha":
    wd_Chrome.find_element(By.XPATH,'//*[@id="live-table"]/div[1]/div[2]/div/button[3]').send_keys(Keys.ENTER)
    time.sleep(2)

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

# Separando apenas os IDs de jogos que eu não peguei ainda
if len(existentes) > 0:
    id_jogos = [id for id in id_jogos if id not in existentes]

base_jogos = pd.DataFrame({})

for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/resumo-de-jogo/resumo-de-jogo')
    
    Odds_H = 0
    Odds_A = 0
    Over_Line = 0
    Odds_Over = 0
    Odds_Under = 0
    HA_Line = 0
    HA_Odds_H = 0
    HA_Odds_A = 0
    
    # Pegando as Informacoes Básicas do Jogo
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text

        Country = clean_name(Country)
        League = clean_name(League)
        League_name = Country + ' ' + League

        # if League_name not in lista_ligas:
        #     # Condição satisfeita, pula para a próxima iteração do loop
        #     continue
    
        # Match Odds
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/home-away/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
            bookie = bookie.get_attribute('title')
            if ((bookie == 'bet365')):
                Odds_H = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odds_A = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                break
            else:
                pass

        if Odds_H == 0:
            continue

        # Over/Under
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/acima-abaixo/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')

        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            Odds_Over = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            Over_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
            if (((bookie == 'bet365') and (Odds_Over >= 1.80) and find_point_five(Over_Line))):
                Odds_Under = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                Over_Line, Odds_Over, Odds_Under = 0, 0, 0                   
                pass


        # Handicap
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/handicap-asiatico/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')

        for celula in celulas:
            
            HA_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
            HA_Odds_H = float(celula.find_elements(By.TAG_NAME,'span')[1].text)

            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            if ((bookie == 'bet365' and HA_Odds_H >= 1.80) and (bookie == 'bet365' and HA_Odds_H <= 2.10) and (find_point_five(HA_Line))):
                HA_Odds_A = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass

        # print(Date, Home, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A)
        # print()

        base_jogos.loc[base_jogos.shape[0],['Fixture ID', 'Date','League','Time','Home','Away','Odds_H','Odds_A','Over_Line','Odds_Over','Odds_Under','HA_Line','HA_Odds_H','HA_Odds_A']] = [
            link, Date, League_name, Time, Home, Away, Odds_H, Odds_A, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A
        ]
    except:
        continue

wd_Chrome.quit()

100%|██████████| 353/353 [14:31<00:00,  2.47s/it]


In [7]:
backup = base_jogos.copy()

# Remove jogos sem odds
base_jogos = base_jogos[base_jogos['Odds_H'] != 0]
base_jogos.reset_index(drop=True, inplace=True)
# Ajusta coluna de data
base_jogos['Date'] = pd.to_datetime(base_jogos['Date'], format='%d.%m.%Y')
base_jogos['Date'] = pd.to_datetime(base_jogos['Date']).dt.date

dia_jogos = base_jogos['Date'].iloc[0]

try:
    existente = pd.read_csv(f'jogos_do_dia/{dia_jogos}.csv')
    n_jogos_antes = existente.shape[0]
    existente = pd.concat([existente, base_jogos])
    existente = existente.drop_duplicates(subset=['Home', 'Away'], keep='first')
    existente.reset_index(drop=True, inplace=True)
    n_jogos_depois = existente.shape[0]
    existente.to_csv(f'jogos_do_dia/{dia_jogos}.csv', index=False)
    print(f'{n_jogos_depois - n_jogos_antes} jogos adicionados aos jogos do dia.')
except:
    base_jogos.to_csv(f'jogos_do_dia/{dia_jogos}.csv', index=False)
    display(base_jogos)


,Fixture ID,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
0,nXK9jrtU,2023-11-04,Espanha Acb,14:00,Unicaja,Obradoiro CAB,1.15,5.75,166.5,1.91,1.91,-5.5,1.42,0.00
1,lt7yn2lo,2023-11-04,Espanha Acb,14:00,Zaragoza,Bilbao,1.59,2.50,164.5,1.95,1.87,-3.5,1.91,1.91
2,QcjKAoRo,2023-11-04,Espanha Acb,16:45,Palencia,Basquet Girona,1.74,2.15,160.5,1.91,1.91,3.5,1.42,0.00
3,4lkO95th,2023-11-04,Espanha Acb,16:45,Tenerife,MoraBanc Andorra,1.33,3.40,163.5,1.91,1.91,-7.5,1.95,1.87
4,h4ET5PGm,2023-11-04,Eua Nba,20:00,Orlando Magic,Los Angeles Lakers,2.40,1.63,218.5,1.91,1.91,3.5,1.91,1.91
5,C6UU33o0,2023-11-04,Eua Nba,21:00,Brooklyn Nets,Boston Celtics,4.50,1.22,229.5,1.91,1.91,9.5,1.91,1.91
6,jcTY2N06,2023-11-04,Eua Nba,21:00,Houston Rockets,Sacramento Kings,2.20,1.71,0.0,0.00,0.00,2.5,1.91,1.91
7,pKSx2sGC,2023-11-04,Eua Nba,21:00,Minnesota Timberwolves,Utah Jazz,1.33,3.40,225.5,1.91,1.91,-7.5,1.91,1.91
8,GzJXaMN3,2023-11-04,Itália Liga A,16:30,Pesaro,Basket Napoli,2.05,1.80,163.5,1.91,1.91,-2.5,1.91,1.91
9,2gSdIhWj,2023-11-04,Alemanha Bbl,14:30,Syntainics MBC,Gottingen,1.80,2.05,165.5,1.91,1.91,-1.5,1.91,1.91
